In [25]:
import pandas as pd
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
from scipy.stats import ttest_ind
import math

In [26]:
'''READS IN TEAM STATS FOR AND CORRECTS NAMING ERRORS'''
team_stats1314 = pd.read_csv('./preprocessed/team_stats1314.csv', sep=',')
team_stats1415 = pd.read_csv('./preprocessed/team_stats1415.csv', sep=',')
team_stats1516 = pd.read_csv('./preprocessed/team_stats1516.csv', sep=',')
team_stats1617 = pd.read_csv('./preprocessed/team_stats1617.csv', sep=',')
team_stats1718 = pd.read_csv('./preprocessed/team_stats1718.csv', sep=',')

dfs = [team_stats1314, team_stats1415, team_stats1516, team_stats1617, team_stats1718]

for df in dfs:
    df['Team'] = df.Team.str.replace(' ', '_')
    df['Team'] = df['Team'].str[:-1]

team_stats1415


,Unnamed: 0,Rk,Team,G,MP,FG,FGA,FG%,3P,3PA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,WIN%
0,0,1,Golden_State_Warriors,82,240.6,41.6,87.0,0.478,10.8,27.0,...,10.4,34.3,44.7,27.4,9.3,6.0,14.5,19.9,110.0,81.7
1,1,4,LA_Clippers,82,240.6,39.4,83.3,0.473,10.1,26.9,...,9.6,33.1,42.6,24.8,7.8,5.0,12.3,21.3,106.7,68.3
2,2,9,Dallas_Mavericks,82,242.4,39.7,85.8,0.463,8.9,25.4,...,10.5,31.8,42.3,22.5,8.1,4.5,13.0,20.0,105.2,61.0
3,3,11,Toronto_Raptors,82,242.1,37.9,83.3,0.455,8.9,25.1,...,10.7,30.8,41.5,20.7,7.5,4.4,12.9,20.9,104.0,59.8
4,4,3,Houston_Rockets,82,241.5,37.0,83.3,0.444,11.4,32.7,...,11.7,32.0,43.7,22.2,9.5,5.0,16.7,22.0,103.9,68.3
5,5,5,San_Antonio_Spurs,82,243.4,39.1,83.6,0.468,8.3,22.5,...,9.8,33.8,43.6,24.4,8.0,5.4,14.0,19.1,103.2,67.1
6,6,7,Cleveland_Cavaliers,82,241.2,37.7,82.2,0.458,10.1,27.5,...,11.1,31.9,43.0,22.1,7.4,4.1,14.3,18.4,103.1,64.6
7,7,8,Portland_Trail_Blazers,82,242.1,38.7,86.0,0.450,9.8,27.2,...,10.7,35.1,45.9,21.9,6.4,4.5,13.6,18.2,102.8,62.2
8,8,2,Atlanta_Hawks,82,240.6,38.1,81.7,0.466,10.0,26.2,...,8.7,31.8,40.6,25.7,9.1,4.6,14.2,17.8,102.5,73.2
9,9,16,Boston_Celtics,82,242.4,38.9,87.9,0.443,8.0,24.6,...,11.1,32.7,43.8,24.5,8.2,3.6,13.8,21.2,101.4,48.8


In [61]:
'''READS IN TEAM STATS AGAINST'''
opp_stats1314 = pd.read_csv('./preprocessed/opp_stats1314.csv', sep=',')
opp_stats1415 = pd.read_csv('./preprocessed/opp_stats1415.csv', sep=',')
opp_stats1516 = pd.read_csv('./preprocessed/opp_stats1516.csv', sep=',')
opp_stats1617 = pd.read_csv('./preprocessed/opp_stats1617.csv', sep=',')
opp_stats1718 = pd.read_csv('./preprocessed/opp_stats1718.csv', sep=',')

opp_arr = [opp_stats1314, opp_stats1415, opp_stats1516, opp_stats1617, opp_stats1718]

for df in opp_arr:
    df['Team'] = df['Team'].str[:-1]

In [62]:
'''READS IN SERIES INFORMATION CSV FILES'''
series_in1314 = pd.read_csv('./Playoff_Matchups/Playoff_13-14.csv', sep=',')
series_in1415 = pd.read_csv('./Playoff_Matchups/Playoff_14-15.csv', sep=',')
series_in1516 = pd.read_csv('./Playoff_Matchups/Playoff_15-16.csv', sep=',')
series_in1617 = pd.read_csv('./Playoff_Matchups/Playoff_16-17.csv', sep=',')
series_in1617

,TeamA,TeamB,Winner
0,Boston_Celtics,Chicago_Bulls,0
1,Washington_Wizards,Atlanta_Hawks,0
2,Toronto_Raptors,Milwaukee_Bucks,0
3,Cleveland_Cavaliers,Indiana_Pacers,0
4,Golden_State_Warriors,Portland_Trail_Blazers,0
5,LA_Clippers,Utah_Jazz,1
6,Houston_Rockets,Oklahoma_City_Thunder,0
7,San_Antonio_Spurs,Memphis_Grizzlies,0
8,Boston_Celtics,Washington_Wizards,0
9,Toronto_Raptors,Cleveland_Cavaliers,1


In [63]:
def make_series(tm, series, df):
    
    if tm == 'a':
        a = series['TeamA'].tolist()
    else:
        a = series['TeamB'].tolist()
            
    team = pd.DataFrame()
    
    for x in a:
        to_concat = df[df['Team'] == x]
        frames = [team, to_concat]
        team = pd.concat(frames)
    
    return team
    

In [64]:
'''THIS MAKES THE FOR AND OPPOSING TABLES FOR THE PLAYOFFS TEAM A'''
a = 'a'
team_a1314 = make_series(a, series_in1314, team_stats1314)
team_a_opp1314 = make_series(a, series_in1314, opp_stats1314)
team_a1415 = make_series(a, series_in1415, team_stats1415)
team_a_opp1415 = make_series(a, series_in1415, opp_stats1415)
team_a1516 = make_series(a, series_in1516, team_stats1516)
team_a_opp1516 = make_series(a, series_in1516, opp_stats1516)
team_a1617 = make_series(a, series_in1617, team_stats1617)
team_a_opp1617 = make_series(a, series_in1617, opp_stats1617)

team_a_opp1516


,Unnamed: 0,Team,G,MP,FG,FGA,FG%,3P,3PA,3P%,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
2,2,Cleveland_Cavaliers,82,242.1,36.8,82.1,0.448,7.9,22.7,0.347,...,0.743,9.3,31.8,41.0,21.4,7.2,4.4,13.3,20.6,98.3
4,4,Atlanta_Hawks,82,241.8,37.1,86.1,0.432,8.3,24.5,0.338,...,0.755,11.5,35.0,46.5,22.0,8.6,5.0,16.1,18.3,99.2
3,3,Miami_Heat,82,241.8,37.2,84.3,0.442,7.4,21.2,0.347,...,0.770,9.8,31.5,41.3,20.2,7.5,4.1,12.9,19.6,98.4
1,1,Toronto_Raptors,82,241.2,36.5,82.1,0.444,8.7,23.4,0.373,...,0.748,9.5,31.2,40.8,21.7,6.5,5.4,13.3,22.0,98.2
13,13,Golden_State_Warriors,82,242.4,38.9,89.4,0.435,7.8,23.5,0.332,...,0.757,11.4,32.5,43.9,22.2,8.7,4.1,14.5,19.8,104.1
5,5,LA_Clippers,82,241.8,36.8,84.7,0.434,7.9,23.3,0.338,...,0.751,11.8,34.9,46.7,21.2,7.1,3.2,15.4,22.5,100.2
12,12,Oklahoma_City_Thunder,82,241.8,38.4,87.7,0.438,8.1,23.8,0.342,...,0.756,11.3,28.9,40.2,21.5,8.8,4.5,13.0,20.2,102.9
0,0,San_Antonio_Spurs,82,240.3,35.7,81.8,0.436,6.6,19.9,0.331,...,0.758,9.1,31.4,40.5,20.8,7.2,3.9,14.8,19.5,92.9
2,2,Cleveland_Cavaliers,82,242.1,36.8,82.1,0.448,7.9,22.7,0.347,...,0.743,9.3,31.8,41.0,21.4,7.2,4.4,13.3,20.6,98.3
3,3,Miami_Heat,82,241.8,37.2,84.3,0.442,7.4,21.2,0.347,...,0.770,9.8,31.5,41.3,20.2,7.5,4.1,12.9,19.6,98.4


In [65]:
'''THIS MAKES THE FOR AND OPPOSING TABLES FOR THE PLAYOFFS TEAM B'''
b = 'b'
team_b1314 = make_series(b, series_in1314, team_stats1314)
team_b_opp1314 = make_series(b, series_in1314, opp_stats1314)
team_b1415 = make_series(b, series_in1415, team_stats1415)
team_b_opp1415 = make_series(b, series_in1415, opp_stats1415)
team_b1516 = make_series(b, series_in1516, team_stats1516)
team_b_opp1516 = make_series(b, series_in1516, opp_stats1516)
team_b1617 = make_series(b, series_in1617, team_stats1617)
team_b_opp1617 = make_series(b, series_in1617, opp_stats1617)

team_b1314

,Unnamed: 0,Rk,Team,G,MP,FG,FGA,FG%,3P,3PA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,WIN%
9,9,18,Atlanta_Hawks,82,241.8,37.3,81.6,0.458,9.4,25.8,...,8.7,31.3,40.0,24.9,8.3,4.0,15.3,19.2,101.0,46.3
10,10,14,Washington_Wizards,82,244.6,38.7,84.4,0.459,7.9,20.8,...,10.8,31.4,42.2,23.3,8.1,4.6,14.7,20.4,100.7,53.7
11,11,15,Brooklyn_Nets,82,242.4,35.7,77.9,0.459,8.6,23.4,...,8.8,29.4,38.1,20.9,8.6,3.8,14.5,21.7,98.5,53.7
12,12,16,Charlotte_Bobcats,82,242.7,36.3,82.1,0.442,6.3,17.9,...,9.5,33.2,42.7,21.7,6.1,5.1,12.3,18.2,96.9,52.4
5,5,10,Dallas_Mavericks,82,241.8,39.6,83.6,0.474,8.8,22.9,...,10.2,30.7,40.9,23.6,8.6,4.3,13.5,20.0,104.8,59.8
2,2,5,Portland_Trail_Blazers,82,242.1,39.1,87.0,0.450,9.4,25.3,...,12.5,34.0,46.4,23.2,5.5,4.7,13.7,19.2,106.7,65.9
6,6,8,Golden_State_Warriors,82,241.8,39.5,85.4,0.462,9.4,24.8,...,10.9,34.4,45.3,23.3,7.8,5.0,15.2,21.8,104.3,62.2
14,14,9,Memphis_Grizzlies,82,241.5,38.1,82.0,0.464,4.9,14.0,...,11.6,30.8,42.4,21.9,7.7,4.6,13.7,19.1,96.1,61.0
10,10,14,Washington_Wizards,82,244.6,38.7,84.4,0.459,7.9,20.8,...,10.8,31.4,42.2,23.3,8.1,4.6,14.7,20.4,100.7,53.7
11,11,15,Brooklyn_Nets,82,242.4,35.7,77.9,0.459,8.6,23.4,...,8.8,29.4,38.1,20.9,8.6,3.8,14.5,21.7,98.5,53.7


In [66]:
'''THIS MAKES THE WINNER DATAFRAME'''
winner1314 = series_in1314['Winner']
winner1415 = series_in1415['Winner']
winner1516 = series_in1516['Winner']
winner1617 = series_in1617['Winner']

win1314 = pd.DataFrame(winner1314, columns = ["Winner"])
win1415 = pd.DataFrame(winner1415, columns = ["Winner"])
win1516 = pd.DataFrame(winner1516, columns = ["Winner"])
win1617 = pd.DataFrame(winner1617, columns = ["Winner"])

win1314.head()

,Winner
0,0
1,1
2,1
3,0
4,0


In [67]:
l = [team_a_opp1314, team_a_opp1415, team_a_opp1516, team_a_opp1617, team_b_opp1314, team_b_opp1415, team_b_opp1516, team_b_opp1617]

for j in l:
    del j['Unnamed: 0']
    del j['Team']
    del j['G']
    del j['MP']

team_a_opp1314.columns

Index(['FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'FT', 'FTA',
       'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')

In [68]:
opps = [team_a_opp1314, team_a_opp1415, team_a_opp1516, team_a_opp1617, team_b_opp1314, team_b_opp1415, team_b_opp1516, team_b_opp1617]

for i in opps:
    i.columns = ['FG_opp', 'FGA_opp', 'FG%_opp', '3P_opp', '3PA_opp', '3P%_opp', '2P_opp', '2PA_opp', '2P%_opp', 'FT_opp', 'FTA_opp',
       'FT%_opp', 'ORB_opp', 'DRB_opp', 'TRB_opp', 'AST_opp', 'STL_opp', 'BLK_opp', 'TOV_opp', 'PF_opp', 'PTS_opp']


In [69]:

opp_cols = team_a_opp1314.columns
opp_cols

# team_b1314[col] = list(team_b_opp1314[col])



for col in opp_cols:
    team_a1314[col] = list(team_a_opp1314[col])
    team_a1415[col] = list(team_a_opp1415[col])
    team_a1516[col] = list(team_a_opp1516[col])
    team_a1617[col] = list(team_a_opp1617[col])
    team_b1314[col] = list(team_b_opp1314[col])
    team_b1415[col] = list(team_b_opp1415[col])
    team_b1516[col] = list(team_b_opp1516[col])
    team_b1617[col] = list(team_b_opp1617[col])

team_a1314



,Unnamed: 0,Rk,Team,G,MP,FG,FGA,FG%,3P,3PA,...,FT%_opp,ORB_opp,DRB_opp,TRB_opp,AST_opp,STL_opp,BLK_opp,TOV_opp,PF_opp,PTS_opp
13,13,4,Indiana_Pacers,82,241.2,36.0,80.2,0.449,6.7,18.8,...,0.755,10.4,30.7,41.2,18.6,7.2,4.6,13.7,21.9,92.3
15,15,11,Chicago_Bulls,82,243.0,34.7,80.2,0.432,6.2,17.8,...,0.753,10.7,30.6,41.3,19.6,7.6,6.0,14.4,20.9,91.8
8,8,11,Toronto_Raptors,82,243.4,36.5,81.9,0.445,8.7,23.4,...,0.752,10.4,30.5,41.0,21.0,7.3,4.3,14.9,22.1,98.0
7,7,5,Miami_Heat,82,242.4,38.3,76.5,0.501,8.1,22.3,...,0.750,10.8,29.5,40.3,21.4,8.1,3.1,16.7,20.4,97.4
4,4,1,San_Antonio_Spurs,82,240.9,40.6,83.5,0.486,8.5,21.4,...,0.760,10.5,31.7,42.2,20.4,7.7,4.6,13.9,19.3,97.6
1,1,5,Houston_Rockets,82,241.8,38.0,80.5,0.472,9.5,26.6,...,0.757,11.9,29.7,41.7,22.2,8.6,5.6,14.0,24.3,103.1
0,0,3,LA_Clippers,82,240.9,39.1,82.5,0.474,8.5,24.0,...,0.755,12.3,31.4,43.7,23.3,7.0,3.5,15.4,23.4,101.0
3,3,2,Oklahoma_City_Thunder,82,241.5,39.0,82.7,0.471,8.1,22.4,...,0.727,11.0,30.0,40.9,21.8,8.2,3.6,15.2,20.4,99.8
13,13,4,Indiana_Pacers,82,241.2,36.0,80.2,0.449,6.7,18.8,...,0.755,10.4,30.7,41.2,18.6,7.2,4.6,13.7,21.9,92.3
7,7,5,Miami_Heat,82,242.4,38.3,76.5,0.501,8.1,22.3,...,0.750,10.8,29.5,40.3,21.4,8.1,3.1,16.7,20.4,97.4


In [70]:
team_a = [team_a1314, team_a1415, team_a1516, team_a1617]
team_b = [team_b1314, team_b1415, team_b1516, team_b1617]
arr = team_a + team_b
# team_a

In [71]:
for j in arr:
    del j['Unnamed: 0']
    del j['Team']
#     print(j.dtypes)
#     print(j)


In [72]:


def find_diff(col, cols, year_a, year_b, df):
    col_name = cols[col]
#     print(col_name)
    end = []
    diff_end = []
#     print('hello')
#     print(len(list(year_a[col_name])))
    diff_arr = list(year_a[col_name])
    diff_arr2 = list(year_b[col_name])
#     print(diff_arr)
#     print(diff_arr2)
#     print(len(diff_arr))
    for x in range(len(diff_arr)):
        diff = diff_arr[x] - diff_arr2[x]
        end.append(diff)
#         print(list(end))
#     print(end)
    df[col] = end
    



In [73]:
cols = list(team_a1314.columns)
# print(len(cols))

df1314 = pd.DataFrame()
df1415 = pd.DataFrame()
df1516 = pd.DataFrame()
df1617 = pd.DataFrame()

years = [df1314, df1415, df1516, df1617]

for year in range(len(years)):
    count = 0
    for col in range(len(cols)):
#         print(col)
        find_diff(col, cols, team_a[year], team_b[year], years[year])
        count = count + 1
df1415


,0,1,2,3,4,5,6,7,8,9,...,36,37,38,39,40,41,42,43,44,45
0,-16,0,-2.4,0.7,-1.3,0.015,3.4,6.3,0.049,-2.7,...,0.033,0.2,-0.8,-0.7,-0.2,-0.4,0.4,2.2,-0.4,-3.8
1,-6,0,-0.6,-1.0,0.9,-0.017,1.3,4.0,-0.010,-2.3,...,0.007,0.3,0.3,0.5,-2.1,-1.1,0.6,-5.2,1.1,0.4
2,-9,0,-1.2,-1.2,-5.7,0.015,2.1,2.9,0.040,-3.3,...,-0.010,-0.1,-3.5,-3.6,2.2,0.7,-0.8,-1.8,1.7,-2.5
3,-12,0,-0.6,3.7,4.1,0.021,3.7,7.7,0.028,0.0,...,0.004,1.1,1.9,3.0,0.3,1.5,-2.2,4.0,0.1,1.3
4,3,0,-0.6,0.9,3.5,-0.008,4.6,12.0,0.023,-3.7,...,0.016,0.5,1.7,2.2,0.3,0.6,-1.6,-3.1,-1.6,3.5
5,0,0,-1.3,-0.4,0.5,-0.007,2.8,8.3,-0.008,-3.2,...,-0.005,1.2,-0.5,0.8,2.9,-1.1,0.7,0.4,0.7,3.1
6,-1,0,-2.8,0.3,-0.3,0.005,1.8,4.4,0.009,-1.6,...,-0.004,0.7,0.1,0.8,2.5,-1.2,-1.4,0.1,1.8,3.1
7,-6,0,-0.9,-2.7,-2.5,-0.019,2.5,7.3,-0.004,-5.2,...,-0.002,-0.4,-1.9,-2.3,-0.9,1.6,1.5,0.1,-1.0,-1.8
8,-9,0,-2.8,-0.2,-1.1,0.004,3.9,9.4,0.020,-4.1,...,0.026,1.6,0.4,1.9,3.6,-0.6,0.6,2.1,0.0,-0.7
9,2,0,1.2,-1.1,0.7,-0.016,-2.2,-5.2,-0.014,1.1,...,0.004,0.9,1.4,2.3,-3.9,-0.3,0.9,-1.1,0.7,-0.9


In [74]:
df1314.to_csv('./preprocessed/final/diff1314.csv')
df1415.to_csv('./preprocessed/final/diff1415.csv')
df1516.to_csv('./preprocessed/final/diff1516.csv')
df1617.to_csv('./preprocessed/final/diff1617.csv')

In [75]:
# '''WRITES TEAM A STATS TO CSV FILES'''
# team_a1314.to_csv('./preprocessed/final/playoffs_for_a_1314.csv')
# team_a1415.to_csv('./preprocessed/final/playoffs_for_a_1415.csv')
# team_a1516.to_csv('./preprocessed/final/playoffs_for_a_1516.csv')
# team_a1617.to_csv('./preprocessed/final/playoffs_for_a_1617.csv')

# team_a_opp1314.to_csv('./preprocessed/final/playoffs_opp_a_1314.csv')
# team_a_opp1415.to_csv('./preprocessed/final/playoffs_opp_a_1415.csv')
# team_a_opp1516.to_csv('./preprocessed/final/playoffs_opp_a_1516.csv')
# team_a_opp1617.to_csv('./preprocessed/final/playoffs_opp_a_1617.csv')

In [76]:
# '''WRITES TEAM B STATS TO CSV FILES'''
# team_b1314.to_csv('./preprocessed/final/playoffs_for_b_1314.csv')
# team_b1415.to_csv('./preprocessed/final/playoffs_for_b_1415.csv')
# team_b1516.to_csv('./preprocessed/final/playoffs_for_b_1516.csv')
# team_b1617.to_csv('./preprocessed/final/playoffs_for_b_1617.csv')

# team_b_opp1314.to_csv('./preprocessed/final/playoffs_opp_b_1314.csv')
# team_b_opp1415.to_csv('./preprocessed/final/playoffs_opp_b_1415.csv')
# team_b_opp1516.to_csv('./preprocessed/final/playoffs_opp_b_1516.csv')
# team_b_opp1617.to_csv('./preprocessed/final/playoffs_opp_b_1617.csv')

In [77]:
# '''WRITES WINNER DATAFRAME TO CSV FILES'''
#win1314.to_csv('./preprocessed/final/winner1314.csv')
#win1415.to_csv('./preprocessed/final/winner1415.csv')
#win1516.to_csv('./preprocessed/final/winner1516.csv')
#win1617.to_csv('./preprocessed/final/winner1617.csv')

In [50]:
#team_stats1718 drop unnamed, team, g, mp
del team_stats1718['Unnamed: 0']
del team_stats1718['Team']
del team_stats1718['G']
del team_stats1718['MP']

In [54]:
#opp_stats1718
del opp_stats1718['Unnamed: 0']
del opp_stats1718['Team']
del opp_stats1718['G']
del opp_stats1718['MP']

In [58]:
stats_1718 = team_stats1718 - opp_stats1718

,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,FT,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,37.3,83.0,0.449,9.5,26.0,0.365,27.8,57.0,0.488,15.8,...,0.771,8.7,33.0,41.6,20.0,8.3,4.6,14.9,20.5,99.8
1,38.2,84.3,0.453,9.1,26.3,0.348,29.0,58.0,0.501,14.3,...,0.759,9.2,33.3,42.5,21.8,7.6,3.9,14.1,19.8,99.8
2,37.4,85.0,0.440,9.4,27.7,0.339,28.0,57.4,0.488,16.3,...,0.763,9.6,34.2,43.8,21.3,7.2,4.4,14.1,19.2,100.4
3,37.7,83.8,0.450,9.6,26.6,0.360,28.1,57.2,0.491,18.0,...,0.783,9.1,34.0,43.1,21.0,7.5,4.6,14.1,19.4,102.9
4,38.4,85.9,0.447,9.7,26.6,0.364,28.7,59.3,0.484,16.6,...,0.755,9.3,33.6,42.9,20.2,7.3,5.1,12.6,19.1,103.0
5,39.6,85.6,0.462,10.1,28.8,0.351,29.5,56.7,0.519,14.6,...,0.746,8.7,33.4,42.0,22.4,7.4,4.4,14.6,20.4,103.9
6,38.4,85.6,0.449,8.9,25.0,0.357,29.5,60.6,0.486,18.2,...,0.767,9.8,32.6,42.4,21.8,7.2,4.9,14.3,19.9,103.9
7,39.4,84.8,0.465,10.6,30.4,0.349,28.8,54.3,0.530,14.8,...,0.757,10.2,32.8,43.0,23.2,7.0,5.2,15.5,18.5,104.2
8,38.5,84.0,0.458,11.2,30.6,0.367,27.3,53.4,0.511,16.2,...,0.769,9.5,32.6,42.2,23.3,7.7,4.6,16.0,21.3,104.4
9,38.0,87.6,0.434,10.1,29.6,0.342,27.9,58.0,0.482,19.1,...,0.745,10.0,32.2,42.2,21.7,8.5,5.1,14.3,20.4,105.3


In [56]:
team_stats1718

,Rk,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,WIN%
0,3,42.8,85.1,0.503,11.3,28.9,0.391,31.5,56.2,0.560,...,8.4,35.1,43.5,29.3,8.0,7.5,15.5,19.6,113.5,70.7
1,1,38.7,84.2,0.460,15.3,42.3,0.362,23.4,41.9,0.558,...,9.0,34.5,43.5,21.5,8.5,4.8,13.8,19.5,112.4,79.3
2,8,42.7,88.3,0.483,10.2,28.2,0.362,32.5,60.1,0.540,...,8.7,35.7,44.3,26.8,8.0,5.9,14.9,19.1,111.7,58.5
3,2,41.3,87.4,0.472,11.8,33.0,0.358,29.5,54.5,0.541,...,9.8,34.2,44.0,24.3,7.6,6.1,13.4,21.8,111.7,72.0
4,6,40.4,84.8,0.476,12.0,32.1,0.372,28.4,52.6,0.540,...,8.5,33.7,42.1,23.4,7.1,3.8,13.7,18.6,110.9,61.0
5,5,40.8,86.6,0.472,11.0,29.8,0.369,29.9,56.7,0.526,...,10.9,36.5,47.4,27.1,8.3,5.1,16.5,22.1,109.8,63.4
6,12,41.0,86.1,0.476,8.0,22.5,0.357,33.0,63.6,0.519,...,10.3,31.6,42.0,22.7,8.4,4.2,12.5,18.2,109.5,57.3
7,8,39.9,88.0,0.453,10.7,30.4,0.354,29.1,57.7,0.505,...,12.5,32.6,45.1,21.3,9.1,5.0,14.0,20.2,107.9,58.5
8,17,39.9,85.6,0.467,9.9,26.5,0.375,30.0,59.1,0.508,...,10.0,33.1,43.1,25.2,7.9,4.3,14.6,21.3,106.6,52.4
9,15,39.7,83.0,0.478,8.8,24.7,0.355,31.0,58.3,0.531,...,8.4,31.5,39.8,23.2,8.8,5.4,13.8,21.4,106.5,53.7


In [57]:
opp_stats1718

,FG,FGA,FG%,3P,3PA,3P%,2P,2PA,2P%,FT,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,37.3,83.0,0.449,9.5,26.0,0.365,27.8,57.0,0.488,15.8,...,0.771,8.7,33.0,41.6,20.0,8.3,4.6,14.9,20.5,99.8
1,38.2,84.3,0.453,9.1,26.3,0.348,29.0,58.0,0.501,14.3,...,0.759,9.2,33.3,42.5,21.8,7.6,3.9,14.1,19.8,99.8
2,37.4,85.0,0.440,9.4,27.7,0.339,28.0,57.4,0.488,16.3,...,0.763,9.6,34.2,43.8,21.3,7.2,4.4,14.1,19.2,100.4
3,37.7,83.8,0.450,9.6,26.6,0.360,28.1,57.2,0.491,18.0,...,0.783,9.1,34.0,43.1,21.0,7.5,4.6,14.1,19.4,102.9
4,38.4,85.9,0.447,9.7,26.6,0.364,28.7,59.3,0.484,16.6,...,0.755,9.3,33.6,42.9,20.2,7.3,5.1,12.6,19.1,103.0
5,39.6,85.6,0.462,10.1,28.8,0.351,29.5,56.7,0.519,14.6,...,0.746,8.7,33.4,42.0,22.4,7.4,4.4,14.6,20.4,103.9
6,38.4,85.6,0.449,8.9,25.0,0.357,29.5,60.6,0.486,18.2,...,0.767,9.8,32.6,42.4,21.8,7.2,4.9,14.3,19.9,103.9
7,39.4,84.8,0.465,10.6,30.4,0.349,28.8,54.3,0.530,14.8,...,0.757,10.2,32.8,43.0,23.2,7.0,5.2,15.5,18.5,104.2
8,38.5,84.0,0.458,11.2,30.6,0.367,27.3,53.4,0.511,16.2,...,0.769,9.5,32.6,42.2,23.3,7.7,4.6,16.0,21.3,104.4
9,38.0,87.6,0.434,10.1,29.6,0.342,27.9,58.0,0.482,19.1,...,0.745,10.0,32.2,42.2,21.7,8.5,5.1,14.3,20.4,105.3
